In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv("./CSV/continents2.csv")

In [3]:
df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [4]:
df.keys()

Index(['name', 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region',
       'sub-region', 'intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code'],
      dtype='object')

In [5]:
continent_df = df[['name','alpha-2','alpha-3','region','sub-region','country-code','region-code','sub-region-code']]

In [6]:
continent_df.columns = ["country", "iso_a2", "iso_a3","continent", "subregion", "country_code", "continent_code", "subregion_code"]

In [7]:
continent_df.head()

,country,iso_a2,iso_a3,continent,subregion,country_code,continent_code,subregion_code
0,Afghanistan,AF,AFG,Asia,Southern Asia,4,142.0,34.0
1,Åland Islands,AX,ALA,Europe,Northern Europe,248,150.0,154.0
2,Albania,AL,ALB,Europe,Southern Europe,8,150.0,39.0
3,Algeria,DZ,DZA,Africa,Northern Africa,12,2.0,15.0
4,American Samoa,AS,ASM,Oceania,Polynesia,16,9.0,61.0


In [8]:
continent_df.isnull().any()

country           False
iso_a2             True
iso_a3            False
continent          True
subregion          True
country_code      False
continent_code     True
subregion_code     True
dtype: bool

In [9]:
continent_df.set_index("iso_a3", inplace=True)

In [10]:
continent_df.head()

,country,iso_a2,continent,subregion,country_code,continent_code,subregion_code
iso_a3,,,,,,,
AFG,Afghanistan,AF,Asia,Southern Asia,4,142.0,34.0
ALA,Åland Islands,AX,Europe,Northern Europe,248,150.0,154.0
ALB,Albania,AL,Europe,Southern Europe,8,150.0,39.0
DZA,Algeria,DZ,Africa,Northern Africa,12,2.0,15.0
ASM,American Samoa,AS,Oceania,Polynesia,16,9.0,61.0


### Connection to database

In [11]:
rds_connection_string = "postgres:postgres@localhost:5432/COVID_df"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
engine.table_names()

['happiness_table',
 'countrycode_table',
 'covid_cases',
 'capital_table',
 'completed_table']

In [13]:
continent_df.to_sql(name='countrycode_table', con=engine, if_exists='append', index=True, index_label="iso_a3")

In [14]:
pd.read_sql_query('select * from countrycode_table', con=engine).head()

,iso_a3,iso_a2,country,continent,subregion,country_code,continent_code,subregion_code
0,AFG,AF,Afghanistan,Asia,Southern Asia,4,142.0,34.0
1,ALA,AX,Åland Islands,Europe,Northern Europe,248,150.0,154.0
2,ALB,AL,Albania,Europe,Southern Europe,8,150.0,39.0
3,DZA,DZ,Algeria,Africa,Northern Africa,12,2.0,15.0
4,ASM,AS,American Samoa,Oceania,Polynesia,16,9.0,61.0
